In [2]:
import pandas as pd
import numpy as np
import random
import os, time
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

import argparse
import pickle
from collections import namedtuple
from collections import deque
from itertools import count

import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
from torch.distributions import Normal, Categorical
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler

from utils.replaybuffer import PrioritizedBuffer
from model import DuelingDQN

In [ ]:
class DQNAgent:

    def __init__(self, learning_rate=3e-4, gamma=0.99, buffer_size=10000):
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.replay_buffer = PrioritizedBuffer(buffer_size)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model1 = DuelingDQN(obs_shape=, ).to(self.device)
        self.model2 = DuelingDQN(obs_shape=, ).to(self.device)

        self.optimizer1 = torch.optim.Adam(self.model1.parameters())
        self.optimizer2 = torch.optim.Adam(self.model2.parameters())
        self.MSE_loss = nn.MSELoss()
        
    def get_action(self, state, eps=0.05):
        state = torch.FloatTensor(state).float().unsqueeze(0).to(self.device)
        qvals = self.model.forward(state) #list[tensor]
        action = []
        #这里还需要获得每一种动作的离散化后的列表，将argmax后得到的index放入列表中采样
        #假设叫做action_list
        if(np.random.randn() < eps):
            for i in range(len(qvals)):
                action.append(action_list[i][random.randint(0, len(action_list[i])-1)])
        else:
            for i in range(len(qvals)):
                action.append(action_list[i][np.argmax(qvals[i].cpu().detach().numpy())])

        return action

    def compute_td(self, batch_size):
        transitions, idxs, IS_weights = self.replay_buffer.sample(batch_size)
        states, actions, rewards, next_states, dones = transitions
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        IS_weights = torch.FloatTensor(IS_weights).to(self.device)
        done_mask = torch.abs(dones - 1).reshape(batch_size,1)

        actions = actions.view(actions.size(0), 1)
        dones = dones.view(dones.size(0), 1)
        
        curr_Q1 = self.model1.forward(states).gather(1, actions)
        curr_Q2 = self.model2.forward(states).gather(1, actions)
        
        next_Q1 = self.model1.forward(next_states)
        next_Q2 = self.model2.forward(next_states)
        next_Q = torch.min(
            torch.max(self.model1.forward(next_states),1)[0]
            torch.max(self.model2.forward(next_states),1)[0]
        )
        next_Q = next_Q.view(next_Q.size(0), 1)
        expected_Q = rewards.squeeze(1) + self.gamma * max_next_Q * done_mask
        
        td_errors1 = torch.pow(curr_Q1 - expected_Q, 2) * IS_weights
        td_errors2 = torch.pow(curr_Q2 - expected_Q, 2) * IS_weights
        
        return td_errors1, td_errors2, idxs

    def update(self, batch_size):
        td_errors1, td_errors2, idxs = self.compute_td(batch_size)

        # update model
        td_errors_mean1 = td_errors1.mean()
        td_errors_mean2 = td_errors2.mean()
        
        self.optimizer1.zero_grad()
        td_errors_mean1.backward()
        self.optimizer1.step()
        
        self.optimizer2.zero_grad()
        td_errors_mean2.backward()
        self.optimizer2.step()

        # update priorities
        for idx, td_error in zip(idxs, td_errors1.cpu().detach().numpy()):
            self.replay_buffer.update_priority(idx, td_error1)
        for idx, td_error in zip(idxs, td_errors1.cpu().detach().numpy()):
            self.replay_buffer.update_priority(idx, td_error1)

In [ ]:
MAX_EPISODES = 1000
MAX_STEPS = 500
BATCH_SIZE = 32

In [ ]:
# 暂时没有交互，只有从buffer中采样，学习
# 把data用循环存入buffer中
agent.replay_buffer.push(state, action, reward, next_state, done)
for _ in range(MAX_EPISODES):
    agent.update(BATCH_SIZE)

torch.save(agent.model,'checkpoint/DQN.pt')
agent.model.load_state_dict(torch.load('checkpoint/DQN.pt'))
    
# episode_rewards = []

# for episode in range(MAX_EPISODES):
#     state = env.reset()
#     episode_reward = 0

#     for step in range(max_steps):
#         action = agent.get_action(state)
#         next_state, reward, done, _ = env.step(action)
#         agent.replay_buffer.push(state, action, reward, next_state, done)
#         episode_reward += reward

#         if len(agent.replay_buffer) > BATCH_SIZE:
#             agent.update(BATCH_SIZE)   

#         if done or step == MAX_STEPS-1:
#             episode_rewards.append(episode_reward)
#             print("Episode " + str(episode) + ": " + str(episode_reward))
#             break

#         state = next_state

In [5]:
torch.onnx.export(agent,
                  (state),
                  "F:\TX_Distribution\Deploy\BVR\AIPilots\Intelligame\model.onnx", 
                   export_params=True,        # 是否保存训练好的参数在网络中
                   opset_version=10,          # ONNX算子版本
                   do_constant_folding=True,  # 是否不保存常数输出（优化选项）
                   input_names = ['input0'],   
                   output_names = ['output0', 'output1', 'output2', 'output3', 'output4', 'output5', 'output6'])

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3742748864.py, line 3)

In [6]:
data = pd.read_csv('F:/TX_Distribution/Deploy/BVR/save_observation/data',header=None)

ParserError: Error tokenizing data. C error: Expected 616 fields in line 8543, saw 617
